In [41]:
import pandas as pd
import numpy as np

# Get data

In [42]:
questions_df = pd.read_csv("New_grader/quantum_bank.csv")
questions = list(questions_df["Question"])
answers = list(questions_df["Answer"])
hint1 = list(questions_df["Hint1"])
hint2 = list(questions_df["Hint2"])
hint3 = list(questions_df["Hint3"])
index = list(questions_df["Index"])
hints = hint1 + hint2 + hint3

# BART

In [43]:
# Use a pipeline as a high-level helper
from transformers import pipeline
pipe = pipeline("summarization", model="facebook/bart-large-cnn",)

In [57]:
from transformers import BartTokenizer, BartForConditionalGeneration

In [65]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

input_text = hint1[1]

# Tokenize input text and calculate its length
inputs = tokenizer(input_text, return_tensors='pt', max_length=1024, truncation=True)
input_length = len(inputs['input_ids'][0])

# Set max_length for summarization dynamically
# For example, setting it to half of the input length
max_length_for_summarization = max(12, min(22, input_length // 2))

# Generate summary
summary_ids = model.generate(inputs['input_ids'], max_length=len(inputs['input_ids'][0]))
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print(summary)
print(hint1[1])

Show that the probability of getting spin up (along b) in the first measurement is P_+=cos^2(\theta_{ab}/2) Find the other three probabilities (P+−, P++, and P−+) Beware: If the outcome of the first measurements was spin down, the relevant angle is now the supplement of θbc.
Show that the probability of getting spin up (along b) in the first measurement is P_+=cos^2(\theta_{ab}/2), and (by extension) the probability of getting spin up in both measurements is P_{++}=cos^2(\theta_{ab}/2)cos^2(\theta_{bc}/2). Find the other three probabilities (P_{+−}, P_{−+}, and P_{−−}). Beware: If the outcome of the first measurement was spin down, the relevant angle is now the supplement of θbc.


In [68]:

def get_summary(hints):
    summaries = []
    for idx,hint in enumerate(hints):
        hint_token = tokenizer(hint, return_tensors='pt', max_length=1024, truncation=True)
        hint_len   = len(hint_token['input_ids'][0])
        summary_ids = model.generate(hint_token['input_ids'], max_length=hint_len)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary)
    return summaries

hint1_summary = get_summary(hint1)
hint2_summary = get_summary(hint2)
hint3_summary = get_summary(hint3)


In [69]:
df = pd.DataFrame({
    "Question":questions,
    "Answer":answers,
    "Hint1":hint1,
    "Hint1_summary":hint1_summary,
    "Hint2":hint2,
    "Hint2_summary":hint2_summary,
    "Hint3":hint3,
    "Hint3_summary":hint3_summary,
    "Index":index
})

In [70]:
df.to_csv("New_grader/quantum_bank_summary.csv",index=False)

In [76]:
import json
f_sum = open("New_grader/auto_graded_GPT4_sum.json")
f     = open("New_grader/auto_graded_GPT4.json")
sum_result = json.load(f_sum)
result     = json.load(f)

In [79]:
for i in range(len(result)):
    print("Question: ",i+1)
    no_hint = result[i]["No_hint"]['grade']
    hint1   = result[i]["Hint1"]['grade']
    hint2   = result[i]["Hint2"]['grade']
    hint3   = result[i]["Hint3"]['grade']
    sum_no_hint = sum_result[i]["No_hint"]['grade']
    sum_hint1   = sum_result[i]["Hint1_summary"]['grade']
    sum_hint2   = sum_result[i]["Hint2_summary"]['grade']
    sum_hint3   = sum_result[i]["Hint3_summary"]['grade']
    print("No hint: ",no_hint," No hint Summary: ",sum_no_hint)
    print("Hint1: ",hint1," Hint1 Summary: ",sum_hint1)
    print("Hint2: ",hint2," Hint2 Summary: ",sum_hint2)
    print("Hint3: ",hint3," Hint3 Summary: ",sum_hint3)

Question:  1
No hint:  7  No hint Summary:  7
Hint1:  7  Hint1 Summary:  7
Hint2:  6  Hint2 Summary:  4
Hint3:  7  Hint3 Summary:  10
Question:  2
No hint:  6  No hint Summary:  6
Hint1:  10  Hint1 Summary:  7
Hint2:  7  Hint2 Summary:  9
Hint3:  7  Hint3 Summary:  10
Question:  3
No hint:  6  No hint Summary:  7
Hint1:  7  Hint1 Summary:  7
Hint2:  7  Hint2 Summary:  7
Hint3:  7  Hint3 Summary:  7
Question:  4
No hint:  10  No hint Summary:  10
Hint1:  7  Hint1 Summary:  10
Hint2:  10  Hint2 Summary:  10
Hint3:  7  Hint3 Summary:  9
Question:  5
No hint:  9  No hint Summary:  10
Hint1:  10  Hint1 Summary:  10
Hint2:  7  Hint2 Summary:  7
Hint3:  9  Hint3 Summary:  7
Question:  6
No hint:  7  No hint Summary:  4
Hint1:  10  Hint1 Summary:  7
Hint2:  10  Hint2 Summary:  7
Hint3:  7  Hint3 Summary:  4
Question:  7
No hint:  6  No hint Summary:  7
Hint1:  7  Hint1 Summary:  7
Hint2:  7  Hint2 Summary:  6
Hint3:  7  Hint3 Summary:  7
Question:  8
No hint:  4  No hint Summary:  5
Hint1:  6 